In [1]:
import pandas as pd
import requests

# 1. 동탄역 정차 노선 목록
> 교통카드 빅데이터 시스템

In [11]:
df_east = pd.read_csv('C:/Users/OneYoung/Documents/GitHub/GTX-A-Bus-lane-optimization/GTX_동탄역_data/교통카드 빅데이터/동탄역(동측)_노선·정류장 지표(정류장별 이용량)_20240617.csv', encoding = 'euc-kr')
df_west = pd.read_csv('C:/Users/OneYoung/Documents/GitHub/GTX-A-Bus-lane-optimization/GTX_동탄역_data/교통카드 빅데이터/동탄역(서측)_노선·정류장 지표(정류장별 이용량)_20240617.csv', encoding = 'euc-kr')

In [18]:
df_east.head(2)

,정류장명,정류장번호,노선,기종점,일자,이용자유형,04(승차),04(하차),05(승차),05(하차),...,23(하차),00(승차),00(하차),01(승차),01(하차),02(승차),02(하차),03(승차),03(하차),Unnamed: 54
0,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,일반인,3,0,7,0,...,7,0,15,0,0,0,0,0,0,NaN
1,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,어린이,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [20]:
#동탄역(동측+서측)에 정차하는 노선 목록 확인
east = df_east['노선'].unique().tolist()
west = df_west['노선'].unique().tolist()
bus_lane = (east+west)

In [23]:
#노선수
len(bus_lane)

62

# 2. 세대 현황
> 행정안전부

In [34]:
toll = pd.read_excel('C:/Users/OneYoung/Documents/GitHub/GTX-A-Bus-lane-optimization/GTX_동탄역_data/행정안전부/행안부_202405_202405_주민등록인구및세대현황_월간.xlsx')
# 특정 행을 컬럼으로 설정하기
toll.columns = toll.iloc[1]

# 불필요한 행 제거 (첫 두 행 제거)
toll = toll[2:]

# 인덱스 리셋 (기존 인덱스는 드롭)
toll = toll.reset_index(drop=True)

toll.head(4)

c:\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1,행정기관코드,행정기관,총 거주자수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,4159000000,경기도 경기도 화성시,"949,720","407,372",2.33,"492,894","456,826",1.08
1,4159025300,경기도 화성시 봉담읍,"98,700","41,334",2.39,"49,906","48,794",1.02
2,4159025600,경기도 화성시 우정읍,"17,275","9,307",1.86,"9,508","7,767",1.22
3,4159025900,경기도 화성시 향남읍,"85,845","40,355",2.13,"46,143","39,702",1.16


In [35]:
toll['행정기관'].unique()

array(['경기도 경기도 화성시 ', '경기도 화성시 봉담읍', '경기도 화성시 우정읍', '경기도 화성시 향남읍',
       '경기도 화성시 남양읍', '경기도 화성시 매송면', '경기도 화성시 비봉면', '경기도 화성시 마도면',
       '경기도 화성시 송산면', '경기도 화성시 서신면', '경기도 화성시 팔탄면', '경기도 화성시 장안면',
       '경기도 화성시 양감면', '경기도 화성시 정남면', '경기도 화성시 새솔동', '경기도 화성시 진안동',
       '경기도 화성시 병점1동', '경기도 화성시 병점2동', '경기도 화성시 반월동', '경기도 화성시 기배동',
       '경기도 화성시 화산동', '경기도 화성시 동탄1동', '경기도 화성시 동탄2동', '경기도 화성시 동탄3동',
       '경기도 화성시 동탄4동', '경기도 화성시 동탄5동', '경기도 화성시 동탄6동', '경기도 화성시 동탄7동',
       '경기도 화성시 동탄8동', '경기도 화성시 동탄9동'], dtype=object)

# 3. 버스 정류장 주소